<a id='top'></a>

# Data Parsing of StatsBomb World Cup Data
##### Notebook to parse 2018 FIFA World Cup JSON data from the [StatsBomb Open Data GitHub repository](https://github.com/statsbomb/open-data) to create one unified Events data DataFrame.

### By [Edd Webster](https://www.twitter.com/eddwebster)
Notebook first written: 06/07/2020<br>
Notebook last updated: 09/07/2021

![StatsBomb](../../img/logos/stats-bomb-logo.png)

![title](../../img/fifaworldcup2018.jpg)

Click [here](#section5) to jump straight to the Exploratory Data Analysis section and skip the [Task Brief](#section2), [Data Sources](#section3), and [Data Engineering](#section4) sections. Or click [here](#section6) to jump straight to the Conclusion.

___


## <a id='import_libraries'>Introduction</a>
This notebook parses pubicly available [StatsBomb](https://statsbomb.com/) Event data, using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

For more information about this notebook and the author, I'm available through all the following channels:
*    [eddwebster.com](https://www.eddwebster.com/);
*    edd.j.webster@gmail.com;
*    [@eddwebster](https://www.twitter.com/eddwebster);
*    [linkedin.com/in/eddwebster](https://www.linkedin.com/in/eddwebster/);
*    [github/eddwebster](https://github.com/eddwebster/);
*    [public.tableau.com/profile/edd.webster](https://public.tableau.com/profile/edd.webster);
*    [kaggle.com/eddwebster](https://www.kaggle.com/eddwebster); and
*    [hackerrank.com/eddwebster](https://www.hackerrank.com/eddwebster).

![title](../../img/edd_webster/fifa21eddwebsterbanner.png)

The accompanying GitHub repository for this notebook can be found [here](https://github.com/eddwebster/football_analytics) and a static version of this notebook can be found [here](https://nbviewer.jupyter.org/github/eddwebster/football_analytics/blob/master/notebooks/2_data_parsing/StatsBomb%20Parsing%20and%20Data%20Engineering%202018%20FIFA%20World%20Cup.ipynb).

___

## <a id='notebook_contents'>Notebook Contents</a>
1.    [Notebook Dependencies](#section1)<br>
2.    [Project Brief](#section2)<br>
3.    [Data Sources](#section3)<br>
      1.    [Introduction](#section3.1)<br>
      2.    [Read in the Datasets](#section3.2)<br>
      3.    [Join the Datasets](#section3.3)<br>
      4.    [Initial Data Handling](#section3.4)<br>
4.    [Data Engineering](#section4)<br>
      1.    [Assign Raw DataFrame to Engineered DataFrame](#section4.1)<br>
      2.    [Sort the DataFrame](#section4.2)<br>
      3.    [Create New Attributes](#section4.3)<br>
      4.    [Fill Null Values](#section4.4)<br>
      5.    [Determine Each Player's Most Frequent Position](#section4.5)<br>
5.    [Aggregated Data](#section5)<br>
      1.    [Groupby and Aggregate by Player and Match](#section5.1)<br>
      2.    [Groupby and Aggregate by Player for the Entire Tournament¶](#section5.2)<br>
6.    [Subset Data](#section6)<br>
7.    [Summary](#section7)<br>
8.    [Next Steps](#section8)<br>
9.    [References](#section9)<br>

___

<a id='section1'></a>

## <a id='#section1'>1. Notebook Dependencies</a>

This notebook was written using [Python 3](https://docs.python.org/3.7/) and requires the following libraries:
*    [`Jupyter notebooks`](https://jupyter.org/) for this notebook environment with which this project is presented;
*    [`NumPy`](http://www.numpy.org/) for multidimensional array computing;
*    [`pandas`](http://pandas.pydata.org/) for data analysis and manipulation; and
*    `tqdm` for a clean progress bar;

All packages used for this notebook except for BeautifulSoup can be obtained by downloading and installing the [Conda](https://anaconda.org/anaconda/conda) distribution, available on all platforms (Windows, Linux and Mac OSX). Step-by-step guides on how to install Anaconda can be found for Windows [here](https://medium.com/@GalarnykMichael/install-python-on-windows-anaconda-c63c7c3d1444) and Mac [here](https://medium.com/@GalarnykMichael/install-python-on-mac-anaconda-ccd9f2014072), as well as in the Anaconda documentation itself [here](https://docs.anaconda.com/anaconda/install/).

### Import Libraries and Modules

In [1]:
%load_ext autoreload
%autoreload 2

# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd    # version 1.0.3
import os    #  used to read the csv filenames
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os

# Working with JSON
import json
import codecs
from pandas.io.json import json_normalize

# Football Libraries
from FCPython import createPitch

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno as msno    # visually display missing data

# Progress Bar
from tqdm import tqdm    # a clean progress bar library

# Display in Jupyter
from IPython.display import Image, Video, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

print('Setup Complete')

Setup Complete


In [2]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))
print('Seaborn: {}'.format(sns.__version__))

Python: 3.7.6
NumPy: 1.18.0
pandas: 1.2.0
matplotlib: 3.3.2
Seaborn: 0.11.1


### Defined Variables

In [3]:
# Define today's date
today = datetime.datetime.now().strftime('%d/%m/%Y').replace('/', '')

### Defined Filepaths

In [4]:
# Set up initial paths to subfolders
base_dir = os.path.join('..', '..', )
data_dir = os.path.join(base_dir, 'data')
data_dir_sb = os.path.join(base_dir, 'data', 'sb')
scripts_dir = os.path.join(base_dir, 'scripts')
scripts_dir_sb = os.path.join(base_dir, 'scripts', 'sb')
data_dir_understat = os.path.join(base_dir, 'data', 'understat')
img_dir = os.path.join(base_dir, 'img')
fig_dir = os.path.join(base_dir, 'img', 'fig')
video_dir = os.path.join(base_dir, 'video')

### Create Directory Structure

In [5]:
# make the directory structure
for folder in ['combined', 'competitions', 'events', 'related', 'freeze', 'tactics', 'lineups']:
    path = os.path.join(data_dir_sb, 'raw', folder)
    if not os.path.exists(path):
        os.mkdir(path)

### Custom Functions

In [6]:
# Define custom function to read JSON files that also handles the encoding of special characters e.g. accents in names of players and teams
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')
    
# Define custom function to flatten pandas DataFrames with nested JSON columns. Source: https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json
def flatten_nested_json_df(df):

    df = df.reset_index()

    print(f"original shape: {df.shape}")
    print(f"original columns: {df.columns}")


    # search for columns to explode/flatten
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()

    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()

    print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []

        for col in dict_columns:
            print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace

        for col in list_columns:
            print(f"exploding: {col}")
            # explode lists vertically, adding new columns
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)

        # check if there are still dict o list fields to flatten
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()

        print(f"lists: {list_columns}, dicts: {dict_columns}")

    print(f"final shape: {df.shape}")
    print(f"final columns: {df.columns}")
    return df

### Notebook Settings

In [7]:
pd.set_option('display.max_columns', None)

---

<a id='section2'></a>

## <a id='#section2'>2. Project Brief</a>
>Norwich City are a possession based team and therefore ball playing centre backs play a vital role within our game model. For the first stage of the interview process you are required to identify the top three ball playing centre backs from the [2018 FIFA World Cup](https://www.fifa.com/worldcup/archive/russia2018). Your analysis should include the selection of relevant position specific metrics/KPI's, data analysis and visualisation to communicate your analysis.
>
>For this practical task you are required to use [StatsBomb](https://statsbomb.com/) 's open data from the FIFA World Cup, 2018 which can be accessed as follows;
>
>*    Sign up to gain access to their open data via the following URL: https://statsbomb.com/academy/
>*    Access all the necessary resources via Statsbomb's GitHub site: https://github.com/statsbomb/open-data
>
>Please use any presentation methods you deem suitable to compete this task. Finally please detail your process and methodology used for this task as an appendix to the main presentation.

<br>

**Notebook Conventions**:<br>
*    Variables that refer a `DataFrame` object are prefixed with `df_`.
*    Variables that refer to a collection of `DataFrame` objects (e.g., a list, a set or a dict) are prefixed with `dfs_`.

---

<a id='section3'></a>

## <a id='#section3'>3. Data Sources</a>

### <a id='#section3.1'>3.1. Introduction</a>

#### <a id='#section3.1.1'>3.1.1. About StatsBomb</a>
[StatsBomb](https://statsbomb.com/) are a football analytics and data company.

![title](../../img/logos/stats-bomb-logo.png)

Before conducting our EDA, the data needs to be imported as a DataFrame in the Data Sources section [Section 3](#section3) and Cleaned in the Data Engineering section [Section 4](#section4).

We'll be using the [pandas](http://pandas.pydata.org/) library to import our data to this workbook as a DataFrame.

#### <a id='#section3.1.2'>3.1.2. About the StatsBomb publicly available data</a>
The complete data set contains:
- 7 competitions;
- 879 matches;
- 3,161,917 events; and
- z players.

The datasets we will be using are:
- competitions;
- matches;
- events;
- lineups; and
- tactics;

The data needs to be imported as a DataFrame in the Data Sources section [Section 3](#section3) and cleaned in the Data Engineering section [Section 4](#section4).

### <a id='#section3.2'>3.2. Read in Data</a>
The following cells read the the `JSON` files into a `DataFrame` object with some basic Data Engineering to flatten the data and select only the columns of interest, to ensure the notebook doesn't crash on a standard laptop.

#### <a id='#section3.3.1.'>3.3.1. Competitions</a>

##### Data dictionary

In [8]:
# ADD MARKDOWN TABLE OF DATA HERE

##### Read in JSON files

In [9]:
# Show files in directory
print(glob.glob(os.path.join(data_dir_sb, 'raw', 'competitions/*')))

['../../data/sb/raw/competitions/competitions_wc2018.csv', '../../data/sb/raw/competitions/competitions.csv']


In [10]:
# Read in exported CSV file if exists, if not, read in JSON file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'competitions', 'competitions_wc2018.csv')):
    json_competitions = read_json_file(os.path.join(data_dir_sb, 'open-data', 'data', 'competitions.json'))
    df_competitions_flat = pd.read_json(json_competitions)
else:
    df_competitions_flat = pd.read_csv(os.path.join(data_dir_sb, 'raw', 'competitions', 'competitions_wc2018.csv'))    
    
# Display DataFrame
df_competitions_flat

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,16,4,Europe,Champions League,male,2018/2019,2020-10-25T12:33:27.855343,2020-10-25T12:33:27.855343
1,16,1,Europe,Champions League,male,2017/2018,2021-01-23T21:55:30.425330,2021-01-23T21:55:30.425330
2,16,2,Europe,Champions League,male,2016/2017,2020-08-26T12:33:15.869622,2020-07-29T05:00
3,16,27,Europe,Champions League,male,2015/2016,2020-08-26T12:33:15.869622,2020-07-29T05:00
4,16,26,Europe,Champions League,male,2014/2015,2020-08-26T12:33:15.869622,2020-07-29T05:00
5,16,25,Europe,Champions League,male,2013/2014,2020-08-26T12:33:15.869622,2020-07-29T05:00
6,16,24,Europe,Champions League,male,2012/2013,2020-08-26T12:33:15.869622,2020-07-29T05:00
7,16,23,Europe,Champions League,male,2011/2012,2020-08-26T12:33:15.869622,2020-07-29T05:00
8,16,22,Europe,Champions League,male,2010/2011,2020-07-29T05:00,2020-07-29T05:00
9,16,21,Europe,Champions League,male,2009/2010,2020-07-29T05:00,2020-07-29T05:00


In [11]:
df_competitions_flat.shape

(37, 8)

##### Identify 2018 FIFA World Cup

In [12]:
# Filter DataFrame for rows where 'competition_name' is equal to 'FIFA World Cup'
df_competitions_flat_wc2018 = df_competitions_flat.loc[df_competitions_flat['competition_name'] == 'FIFA World Cup']

In [13]:
df_competitions_flat_wc2018

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
17,43,3,International,FIFA World Cup,male,2018,2020-10-25T14:03:50.263266,2020-10-25T14:03:50.263266


##### Identify Competitions of Interest by ID

In [14]:
# FA Women's Super League has competition ID 37
competition_id = 43

For our analysis, we want just the 2018 FIFA World Cup which has the competition_id - `43`.

##### Export DataFrame

In [15]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'competitions', 'competitions_wc2018.csv')):
    df_competitions_flat.to_csv(os.path.join(data_dir_sb, 'raw', 'competitions', 'competitions_wc2018.csv'), index=None, header=True)
else:
    pass

#### <a id='#section3.3.2.'>3.3.2. Matches</a>

##### Data Dictionary

In [16]:
# ADD MARKDOWN TABLE OF DATA HERE

##### Define competitions
The following cell lists the competitions to be included in the dataset. Dataset includes data for seven different competitions - 5 domestic and 2 international.

In [17]:
# Define a list to select only the competitions of interest. 

# Define list of competitions
lst_competitions = [#2,     # Premier League
                    #11,    # La Liga
                    #16,    # Champions League
                    #37,    # FA Women's Super League
                    43,    # FIFA World Cup
                    #49,    # NWSL
                    #72,    # Women's World Cup
                   ]

# Flatmap all competition IDs to use all available competitions
#lst_competitions = df_competitions['competition_id'].unique().tolist()

# Display list of competitions
lst_competitions

[43]

In [18]:
len(lst_competitions)

1

##### Read in JSON files

In [19]:
# Show files in directory
print(glob.glob(os.path.join(data_dir_sb, 'raw', 'matches/*')))

['../../data/sb/raw/matches/matches.csv', '../../data/sb/raw/matches/matches_wc2018.csv']


In [20]:
# Temporary solution because the loop below doesn't work

# Read in exported CSV file if exists, if not, read in JSON file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'matches', 'matches_wc2018.csv')):
    
    ## Import all StatsBomb JSON Match data for the World Cup 2018
    with open(data_dir_sb + '/open-data/data/matches/' + str(competition_id) + '/3.json') as f:
        json_sb_match_data_wc_2018 = json.load(f)

    ## Flatten the JSON Events data
    df_matches_flat = json_normalize(json_sb_match_data_wc_2018)

    # Flatten the nested columns
    #df_matches_flat = flatten_nested_json_df(df_matches_flat)

    # Rename columns
    #df_matches_flat.columns = df_matches_flat.columns.str.replace('[.]', '_')    # commented out for now
    
else:    
    df_matches_flat = pd.read_csv(os.path.join(data_dir_sb, 'raw', 'matches', 'matches_wc2018.csv'))
    
    
# Display DataFrame
df_matches_flat.head()

,match_id,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name
0,7581,2018-07-01,20:00:00.000,1,1,available,2020-07-29T05:00,4,43,International,FIFA World Cup,3,2018,785,Croatia,male,NaN,56,Croatia,"[{'id': 307, 'name': 'Zlatko Dalić', 'nickname...",776,Denmark,male,NaN,61,Denmark,"[{'id': 641, 'name': 'Åge Fridtjof Hareide', '...",1.0.2,33,Round of 16,4263.0,Stadion Nizhny Novgorod,188.0,Russia,730.0,N. Pitana,NaN,NaN
1,7549,2018-06-22,17:00:00.000,2,0,available,2020-07-29T05:00,2,43,International,FIFA World Cup,3,2018,775,Nigeria,male,Group D,166,Nigeria,"[{'id': 636, 'name': 'Gernot Rohr', 'nickname'...",793,Iceland,male,Group D,104,Iceland,"[{'id': 648, 'name': 'Heimir Hallgrímsson', 'n...",1.0.2,10,Group Stage,4257.0,Volgograd Arena,188.0,Russia,739.0,M. Conger,NaN,NaN
2,7555,2018-06-24,20:00:00.000,0,3,available,2020-07-29T05:00,2,43,International,FIFA World Cup,3,2018,789,Poland,male,Group H,182,Poland,"[{'id': 542, 'name': 'Adam Nawałka', 'nickname...",769,Colombia,male,Group H,49,Colombia,"[{'id': 634, 'name': 'José Néstor Pekerman', '...",1.0.2,10,Group Stage,4258.0,Kazan' Arena (Kazan'),188.0,Russia,740.0,C. Ramos,147.0,Mexico
3,7529,2018-06-16,21:00:00.000,2,0,available,2020-07-29T05:00,1,43,International,FIFA World Cup,3,2018,785,Croatia,male,Group D,56,Croatia,"[{'id': 307, 'name': 'Zlatko Dalić', 'nickname...",775,Nigeria,male,Group D,166,Nigeria,"[{'id': 636, 'name': 'Gernot Rohr', 'nickname'...",1.0.2,10,Group Stage,4260.0,Stadion Kaliningrad,255.0,International,738.0,Sandro Ricci,NaN,NaN
4,7548,2018-06-22,14:00:00.000,2,0,available,2020-07-29T05:00,2,43,International,FIFA World Cup,3,2018,781,Brazil,male,Group E,31,Brazil,"[{'id': 547, 'name': 'Adenor Leonardo Bacchi',...",795,Costa Rica,male,Group E,54,Costa Rica,"[{'id': 646, 'name': 'Óscar Antonio Ramírez He...",1.0.2,10,Group Stage,4726.0,Saint-Petersburg Stadium,255.0,International,287.0,B. Kuipers,160.0,Netherlands


In [21]:
"""
# Read in exported CSV file if exists, if not, read in JSON file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'matches', 'matches_wc2018.csv')):
    # Loop through the competition files for the selected competition(s)
    # Take the separate JSON files each representing a match for the selected competition(s).
    # The file is called {match_id}.json.
    # Read the corresponding JSON matches files using the auxillary function
    # Read JSON file as a pandas DataFrame
    # Append the DataFrames to a list
    # Finally, concatenate all the separate DataFrames into one DataFrame

    ## Create empty list for DataFrames
    dfs_matches = []

    ## Loop through the competition files for the selected competition(s) and append DataFrame to dfs_matches list
    for competition_id in lst_competitions:
        filepath_competition = data_dir_sb + 'open-data/data/matches/' + str(competition_id)
        filepath_matches = (glob.glob(filepath_competition + '/*.json'))
        for filepath_match in filepath_matches:
            df_match = pd.read_json(filepath_match)
            dfs_matches.append(df_match)

    ## Concatenate DataFrames to one DataFrame
    df_matches = pd.concat(dfs_matches)
    
    # Flatten the nested columns
    df_matches_flat = flatten_nested_json_df(df_matches)
    
    ## Rename columns
    #df_matches_flat.columns = df_matches_flat.columns.str.replace('[.]', '_')    # commented out for now
    
else:    
    df_matches_flat = pd.read_csv(os.path.join(data_dir_sb, 'raw', 'matches', 'matches_wc2018.csv'))
    
    
# Display DataFrame
df_matches_flat.head()
"""

"\n# Read in exported CSV file if exists, if not, read in JSON file\nif not os.path.exists(os.path.join(data_dir_sb, 'raw', 'matches', 'matches_wc2018.csv')):\n    # Loop through the competition files for the selected competition(s)\n    # Take the separate JSON files each representing a match for the selected competition(s).\n    # The file is called {match_id}.json.\n    # Read the corresponding JSON matches files using the auxillary function\n    # Read JSON file as a pandas DataFrame\n    # Append the DataFrames to a list\n    # Finally, concatenate all the separate DataFrames into one DataFrame\n\n    ## Create empty list for DataFrames\n    dfs_matches = []\n\n    ## Loop through the competition files for the selected competition(s) and append DataFrame to dfs_matches list\n    for competition_id in lst_competitions:\n        filepath_competition = data_dir_sb + 'open-data/data/matches/' + str(competition_id)\n        filepath_matches = (glob.glob(filepath_competition + '/*.json'

In [22]:
df_matches_flat.shape

(64, 38)

##### Convert `match_id` column to list
List used as reference of matches to parse for Events, Lineups, and Tactics data - iteration through list comprehension.

In [23]:
df_matches_flat.head()

,match_id,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name
0,7581,2018-07-01,20:00:00.000,1,1,available,2020-07-29T05:00,4,43,International,FIFA World Cup,3,2018,785,Croatia,male,NaN,56,Croatia,"[{'id': 307, 'name': 'Zlatko Dalić', 'nickname...",776,Denmark,male,NaN,61,Denmark,"[{'id': 641, 'name': 'Åge Fridtjof Hareide', '...",1.0.2,33,Round of 16,4263.0,Stadion Nizhny Novgorod,188.0,Russia,730.0,N. Pitana,NaN,NaN
1,7549,2018-06-22,17:00:00.000,2,0,available,2020-07-29T05:00,2,43,International,FIFA World Cup,3,2018,775,Nigeria,male,Group D,166,Nigeria,"[{'id': 636, 'name': 'Gernot Rohr', 'nickname'...",793,Iceland,male,Group D,104,Iceland,"[{'id': 648, 'name': 'Heimir Hallgrímsson', 'n...",1.0.2,10,Group Stage,4257.0,Volgograd Arena,188.0,Russia,739.0,M. Conger,NaN,NaN
2,7555,2018-06-24,20:00:00.000,0,3,available,2020-07-29T05:00,2,43,International,FIFA World Cup,3,2018,789,Poland,male,Group H,182,Poland,"[{'id': 542, 'name': 'Adam Nawałka', 'nickname...",769,Colombia,male,Group H,49,Colombia,"[{'id': 634, 'name': 'José Néstor Pekerman', '...",1.0.2,10,Group Stage,4258.0,Kazan' Arena (Kazan'),188.0,Russia,740.0,C. Ramos,147.0,Mexico
3,7529,2018-06-16,21:00:00.000,2,0,available,2020-07-29T05:00,1,43,International,FIFA World Cup,3,2018,785,Croatia,male,Group D,56,Croatia,"[{'id': 307, 'name': 'Zlatko Dalić', 'nickname...",775,Nigeria,male,Group D,166,Nigeria,"[{'id': 636, 'name': 'Gernot Rohr', 'nickname'...",1.0.2,10,Group Stage,4260.0,Stadion Kaliningrad,255.0,International,738.0,Sandro Ricci,NaN,NaN
4,7548,2018-06-22,14:00:00.000,2,0,available,2020-07-29T05:00,2,43,International,FIFA World Cup,3,2018,781,Brazil,male,Group E,31,Brazil,"[{'id': 547, 'name': 'Adenor Leonardo Bacchi',...",795,Costa Rica,male,Group E,54,Costa Rica,"[{'id': 646, 'name': 'Óscar Antonio Ramírez He...",1.0.2,10,Group Stage,4726.0,Saint-Petersburg Stadium,255.0,International,287.0,B. Kuipers,160.0,Netherlands


In [24]:
# Flatmap all competition IDs to use all available competitions
lst_matches = df_matches_flat['match_id'].tolist()

# Display list of competitions
lst_matches

[7581,
 7549,
 7555,
 7529,
 7548,
 7534,
 7562,
 7565,
 7571,
 7569,
 7568,
 7530,
 7558,
 7583,
 8652,
 7547,
 7535,
 7546,
 7584,
 7536,
 7545,
 7580,
 8650,
 8649,
 8658,
 7561,
 7539,
 7563,
 7578,
 7531,
 7582,
 7552,
 7532,
 7544,
 7553,
 7560,
 7533,
 7564,
 7556,
 7559,
 7579,
 7572,
 7577,
 7543,
 7525,
 7538,
 7576,
 7542,
 7566,
 7557,
 7554,
 7570,
 7585,
 7586,
 8656,
 8651,
 7567,
 7541,
 8655,
 8657,
 7540,
 7551,
 7550,
 7537]

In [25]:
len(lst_matches)

64

##### Export DataFrame

In [26]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'matches', 'matches_wc2018.csv')):
    df_matches_flat.to_csv(os.path.join(data_dir_sb, 'raw', 'matches', 'matches_wc2018.csv'), index=None, header=True)
else:
    pass

#### <a id='#section3.3.3.'>3.3.3. Events</a>

##### Data dictionary

The [StatsBomb](https://statsbomb.com/) dataset has one hundred and fourteen features (columns) with the following definitions and data types:

| Feature     | Data type    |
|------|-----|
| `id`    | `object`
| `index`    | `object`
| `period`    | `object`
| `timestamp`    | `object`
| `minute`    | `object`
| `second`    | `object`
| `possession`    | `object`
| `duration`    | `object`
| `type.id`    | `object`
| `type.name`    | `object`
| `possession_team.id`    | `object`
| `possession_team.name`    | `object`
| `play_pattern.id`    | `object`
| `play_pattern.name`    | `object`
| `team.id`    | `object`
| `team.name`    | `object`
| `tactics.formation`    | `object`
| `tactics.lineup`    | `object`
| `related_events`    | `object`
| `location`    | `object`
| `player.id`    | `object`
| `player.name`    | `object`
| `position.id`    | `object`
| `position.name`    | `object`
| `pass.recipient.id`    | `object`
| `pass.recipient.name`    | `object`
| `pass.length`    | `object`
| `pass.angle`    | `object`
| `pass.height.id`    | `object`
| `pass.height.name`    | `object`
| `pass.end_location`    | `object`
| `pass.type.id`    | `object`
| `pass.type.name`    | `object`
| `pass.body_part.id`    | `object`
| `pass.body_part.name`    | `object`
| `carry.end_location`    | `object`
| `under_pressure`    | `object`
| `duel.type.id`    | `object`
| `duel.type.name`    | `object`
| `out`    | `object`
| `miscontrol.aerial_won`    | `object`
| `pass.outcome.id`    | `object`
| `pass.outcome.name`    | `object`
| `ball_receipt.outcome.id`    | `object`
| `ball_receipt.outcome.name`    | `object`
| `pass.aerial_won`    | `object`
| `counterpress`    | `object`
| `off_camera`    | `object`
| `dribble.outcome.id`    | `object`
| `dribble.outcome.name`    | `object`
| `dribble.overrun`    | `object`
| `ball_recovery.offensive`    | `object`
| `shot.statsbomb_xg`    | `object`
| `shot.end_location`    | `object`
| `shot.outcome.id`    | `object`
| `shot.outcome.name`    | `object`
| `shot.type.id`    | `object`
| `shot.type.name`    | `object`
| `shot.body_part.id`    | `object`
| `shot.body_part.name`    | `object`
| `shot.technique.id`    | `object`
| `shot.technique.name`    | `object`
| `shot.freeze_frame`    | `object`
| `goalkeeper.end_location`    | `object`
| `goalkeeper.type.id`    | `object`
| `goalkeeper.type.name`    | `object`
| `goalkeeper.position.id`    | `object`
| `goalkeeper.position.name`    | `object`
| `pass.straight`    | `object`
| `pass.technique.id`    | `object`
| `pass.technique.name`    | `object`
| `clearance.head`    | `object`
| `clearance.body_part.id`    | `object`
| `clearance.body_part.name`    | `object`
| `pass.switch`    | `object`
| `duel.outcome.id`    | `object`
| `duel.outcome.name`    | `object`
| `foul_committed.advantage`    | `object`
| `foul_won.advantage`    | `object`
| `pass.cross`    | `object`
| `pass.assisted_shot_id`    | `object`
| `pass.shot_assist`    | `object`
| `shot.one_on_one`    | `object`
| `shot.key_pass_id`    | `object`
| `goalkeeper.body_part.id`    | `object`
| `goalkeeper.body_part.name`    | `object`
| `goalkeeper.technique.id`    | `object`
| `goalkeeper.technique.name`    | `object`
| `goalkeeper.outcome.id`    | `object`
| `goalkeeper.outcome.name`    | `object`
| `clearance.aerial_won`    | `object`
| `foul_committed.card.id`    | `object`
| `foul_committed.card.name`    | `object`
| `foul_won.defensive`    | `object`
| `clearance.right_foot`    | `object`
| `shot.first_time`    | `object`
| `pass.through_ball`    | `object`
| `interception.outcome.id`    | `object`
| `interception.outcome.name`    | `object`
| `clearance.left_foot`    | `object`
| `ball_recovery.recovery_failure`    | `object`
| `shot.aerial_won`    | `object`
| `pass.goal_assist`    | `object`
| `pass.cut_back`    | `object`
| `pass.deflected`    | `object`
| `clearance.other`    | `object`
| `pass.outswinging`    | `object`
| `substitution.outcome.id`    | `object`
| `substitution.outcome.name`    | `object`
| `substitution.replacement.id`    | `object`
| `substitution.replacement.name`    | `object`
| `block.deflection`    | `object`
| `block.offensive`    | `object`
| `injury_stoppage.in_chain`    | `object`

For a full list of definitions, see the official documentation [[link](https://statsbomb.com/stat-definitions/)].

##### Read in JSON files

In [27]:
# Show files in directory
print(glob.glob(os.path.join(data_dir_sb, 'raw', 'events/*')))

['../../data/sb/raw/events/events.csv', '../../data/sb/raw/events/events_2018.csv']


In [28]:
# Read in exported CSV file if exists, if not, read in JSON file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'events', 'events_wc2018.csv')):
    # Loop through the matches files for the selected match(es)
    # Take the separate JSON file each representing theevents match for the selected matches.
    # The file is called {match_id}.json.
    # Read the corresponding JSON matches files using the auxillary function
    # Read JSON file as a pandas DataFrame
    # Append the DataFrames to a list
    # Finally, concatenate all the separate DataFrames into one DataFrame

    ## Create empty list for DataFrames
    dfs_events = []

    ## Loop through event files for the selected matches and append DataFrame to dfs_events list
    for match_id in lst_matches:
        with open(data_dir_sb + '/open-data/data/events/' + str(match_id) + '.json') as f:
            event = json.load(f)
           #match_id = str(match_id)
            df_event_flat = json_normalize(event)
            df_event_flat['match_id'] = match_id
            dfs_events.append(df_event_flat)    

    ## Concatenate DataFrames to one DataFrame
    df_events = pd.concat(dfs_events)
    
    # Flatten the nested columns
    df_events_flat = flatten_nested_json_df(df_events)
    
    ## Rename columns
    #df_events_flat.columns = df_events_flat.columns.str.replace('[.]', '_')    
    
else:    
    df_events_flat = pd.read_csv(os.path.join(data_dir_sb, 'raw', 'events', 'events_wc2018.csv'))
    
    
# Display DataFrame
df_events_flat.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


original shape: (227886, 123)
original columns: Index(['level_0', 'id', 'index', 'period', 'timestamp', 'minute', 'second',
       'possession', 'duration', 'type.id',
       ...
       'injury_stoppage.in_chain', 'shot.one_on_one', 'block.save_block',
       'pass.miscommunication', 'bad_behaviour.card.id',
       'bad_behaviour.card.name', 'shot.open_goal', 'shot.deflected',
       'shot.redirect', 'shot.follows_dribble'],
      dtype='object', length=123)
lists: [], dicts: []
final shape: (227886, 123)
final columns: Index(['level_0', 'id', 'index', 'period', 'timestamp', 'minute', 'second',
       'possession', 'duration', 'type.id',
       ...
       'injury_stoppage.in_chain', 'shot.one_on_one', 'block.save_block',
       'pass.miscommunication', 'bad_behaviour.card.id',
       'bad_behaviour.card.name', 'shot.open_goal', 'shot.deflected',
       'shot.redirect', 'shot.follows_dribble'],
      dtype='object', length=123)


,level_0,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,location,player.id,player.name,position.id,position.name,pass.recipient.id,pass.recipient.name,pass.length,pass.angle,pass.height.id,pass.height.name,pass.end_location,pass.body_part.id,pass.body_part.name,pass.type.id,pass.type.name,under_pressure,carry.end_location,pass.outcome.id,pass.outcome.name,pass.aerial_won,duel.type.id,duel.type.name,ball_receipt.outcome.id,ball_receipt.outcome.name,pass.switch,pass.assisted_shot_id,pass.goal_assist,shot.statsbomb_xg,shot.end_location,shot.key_pass_id,shot.outcome.id,shot.outcome.name,shot.body_part.id,shot.body_part.name,shot.type.id,shot.type.name,shot.technique.id,shot.technique.name,shot.freeze_frame,goalkeeper.outcome.id,goalkeeper.outcome.name,goalkeeper.body_part.id,goalkeeper.body_part.name,goalkeeper.type.id,goalkeeper.type.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.technique.id,goalkeeper.technique.name,shot.first_time,counterpress,foul_committed.offensive,foul_won.defensive,pass.cross,goalkeeper.end_location,clearance.aerial_won,dribble.outcome.id,dribble.outcome.name,duel.outcome.id,duel.outcome.name,pass.deflected,block.offensive,block.deflection,dribble.overrun,pass.shot_assist,interception.outcome.id,interception.outcome.name,miscontrol.aerial_won,ball_recovery.recovery_failure,foul_committed.advantage,foul_won.advantage,dribble.nutmeg,shot.aerial_won,pass.backheel,50_50.outcome.id,50_50.outcome.name,ball_recovery.offensive,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,foul_committed.type.id,foul_committed.type.name,pass.through_ball,pass.technique.id,pass.technique.name,foul_committed.card.id,foul_committed.card.name,foul_committed.penalty,foul_won.penalty,match_id,pass.cut_back,injury_stoppage.in_chain,shot.one_on_one,block.save_block,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.open_goal,shot.deflected,shot.redirect,shot.follows_dribble
0,0,0aa135b8-37b4-4482-adc7-f02e85a19bec,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,785,Croatia,1,Regular Play,785,Croatia,4141.0,"[{'player': {'id': 3444, 'name': 'Danijel Suba...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,086b7750-936b-4ffd-9a00-bfd72c7a0f26,2,1,00:00:00.000,0,0,1,0.000,35,Starting XI,785,Croatia,1,Regular Play,776,Denmark,4411.0,"[{'player': {'id': 3815, 'name': 'Kasper Schme...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,53f0c3f9-129e-47b5-ba77-ae9d214df56f,3,1,00:00:00.000,0,0,1,0.000,18,Half Start,785,Croatia,1,Regular Play,785,Croatia,NaN,NaN,[49233ae2-594f-43c9-a58c-a6a0b8f99ee2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,49233ae2-594f-43c9-a58c-a6a0b8f99ee2,4,1,00:00

In [29]:
df_events_flat.shape

(227886, 123)

##### Export DataFrame

In [30]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'events', 'events_2018.csv')):
    df_events_flat.to_csv(os.path.join(data_dir_sb, 'raw', 'events', 'events_2018.csv'), index=None, header=True)
else:
    pass

##### View all formations

In [31]:
# Flatmap all formations
lst_formation = df_events_flat['tactics.formation'].tolist()

# Display list of competitions
lst_formation

[4141.0,
 4411.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

#### <a id='#section3.3.3.'>3.3.4. Lineups</a>

##### Data dictionary

In [32]:
# ADD MARKDOWN TABLE OF DATA HERE

##### Read in JSON files

In [33]:
# Show files in directory
print(glob.glob(os.path.join(data_dir_sb, 'raw', 'lineups/*')))

['../../data/sb/raw/lineups/lineups_wc2018.csv', '../../data/sb/raw/lineups/lineups.csv']


In [34]:
# Read in exported CSV file if exists, if not, read in JSON file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'lineups', 'lineups_wc2018.csv')):
    # Loop through the competition files for the selected competition(s)
    # Take the separate JSON files each representing a match for the selected competition(s).
    # The file is called {match_id}.json.
    # Read the corresponding JSON matches files using the auxillary function
    # Read JSON file as a pandas DataFrame
    # Append the DataFrames to a list
    # Finally, concatenate all the separate DataFrames into one DataFrame

    ## Create empty list for DataFrames
    dfs_lineups = []

    ## Loop through event files for the selected matches and append DataFrame to dfs_lineups list
    for match_id in lst_matches:
        with open(data_dir_sb + '/open-data/data/lineups/' + str(match_id) + '.json') as f:
            lineup = json.load(f)
           #match_id = str(match_id)
            df_lineups_flat = json_normalize(lineup)
            df_lineups_flat['match_id'] = match_id
            dfs_lineups.append(df_lineups_flat)    

    ## Concatenate DataFrames to one DataFrame
    df_lineups = pd.concat(dfs_lineups)

    # Flatten the nested columns
    df_lineups_flat = flatten_nested_json_df(df_lineups)
    
    ## Rename columns
    #df_lineups_flat.columns = df_lineups_flat.columns.str.replace('[.]', '_')    # commented out for now
    
else:    
    df_lineups_flat = pd.read_csv(os.path.join(data_dir_sb, 'raw', 'lineups', 'lineups_wc2018.csv'))
    
    
# Display DataFrame
df_lineups_flat.head()

,index,team_id,team_name,match_id,lineup.player_id,lineup.player_name,lineup.player_nickname,lineup.jersey_number,lineup.country.id,lineup.country.name
0,0,776,Denmark,7581,3027,Mathias Jattah-Njie Jørgensen,Zanka,13,61,Denmark
1,0,776,Denmark,7581,3043,Christian Dannemann Eriksen,Christian Eriksen,10,61,Denmark
2,0,776,Denmark,7581,3815,Kasper Schmeichel,NaN,1,61,Denmark
3,0,776,Denmark,7581,3959,Andreas Christensen,NaN,6,61,Denmark
4,0,776,Denmark,7581,4447,Martin Braithwaite Christensen,Martin Braithwaite,11,61,Denmark


In [35]:
df_lineups_flat.shape

(2034, 10)

##### Export DataFrame

In [36]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'lineups', 'lineups_wc2018.csv')):
    df_lineups_flat.to_csv(os.path.join(data_dir_sb, 'raw', 'lineups', 'lineups_wc2018.csv'), index=None, header=True)
else:
    pass

### <a id='#section3.4'>3.4. Join Datasets</a>
Next, we're required to join the `Matches` DataFrame and the `Players` DataFrame to the `Events` DatFrame. The `Events` data is the base DataFrame in which we join the other tables via `x`, `y`, `z`, `z`, and `z`.

In [37]:
# Read in exported CSV file if exists, if not, merge the individual DataFrames
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'combined', 'combined_wc2018.csv')):
    
    # Join the Matches DataFrame to the Events DataFrame
    df_events_matches = pd.merge(df_events_flat, df_matches_flat, left_on=['match_id'], right_on=['match_id'])

    # Join the Competitions DataFrame to the Events-Matches DataFrame
    df_events_matches_competitions = pd.merge(df_events_matches, df_competitions_flat, left_on=['competition.competition_id', 'season.season_id'], right_on=['competition_id', 'season_id'])
    
else:    
    df_events_matches_competitions = pd.read_csv(os.path.join(data_dir_sb, 'raw', 'combined', 'combined_wc2018.csv'))
    
    
# Display DataFrame
df_events_matches_competitions.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (47,63,72,76,81,82,83,84,88,90,91,92,93,96,97,103,104,106,108,109,110,112,113,114,115,116,118,119,120,121,122,138,145,153,155,157,159) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,level_0,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,location,player.id,player.name,position.id,position.name,pass.recipient.id,pass.recipient.name,pass.length,pass.angle,pass.height.id,pass.height.name,pass.end_location,pass.body_part.id,pass.body_part.name,pass.type.id,pass.type.name,under_pressure,carry.end_location,pass.outcome.id,pass.outcome.name,pass.aerial_won,duel.type.id,duel.type.name,ball_receipt.outcome.id,ball_receipt.outcome.name,pass.switch,pass.assisted_shot_id,pass.goal_assist,shot.statsbomb_xg,shot.end_location,shot.key_pass_id,shot.outcome.id,shot.outcome.name,shot.body_part.id,shot.body_part.name,shot.type.id,shot.type.name,shot.technique.id,shot.technique.name,shot.freeze_frame,goalkeeper.outcome.id,goalkeeper.outcome.name,goalkeeper.body_part.id,goalkeeper.body_part.name,goalkeeper.type.id,goalkeeper.type.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.technique.id,goalkeeper.technique.name,shot.first_time,counterpress,foul_committed.offensive,foul_won.defensive,pass.cross,goalkeeper.end_location,clearance.aerial_won,dribble.outcome.id,dribble.outcome.name,duel.outcome.id,duel.outcome.name,pass.deflected,block.offensive,block.deflection,dribble.overrun,pass.shot_assist,interception.outcome.id,interception.outcome.name,miscontrol.aerial_won,ball_recovery.recovery_failure,foul_committed.advantage,foul_won.advantage,dribble.nutmeg,shot.aerial_won,pass.backheel,50_50.outcome.id,50_50.outcome.name,ball_recovery.offensive,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,foul_committed.type.id,foul_committed.type.name,pass.through_ball,pass.technique.id,pass.technique.name,foul_committed.card.id,foul_committed.card.name,foul_committed.penalty,foul_won.penalty,match_id,pass.cut_back,injury_stoppage.in_chain,shot.one_on_one,block.save_block,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.open_goal,shot.deflected,shot.redirect,shot.follows_dribble,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,0,0aa135b8-37b4-4482-adc7-f02e85a19bec,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,785,Croatia,1,Regular Play,785,Croatia,4141.0,"[{'player': {'id': 3444, 'name': 'Danijel Suba...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-01,20:00:00.000,1,1,available,2020-07-29T05:00,4,43,International,FIFA World Cup,3,2018,785,Croatia,male,NaN,56,Croatia,"[{'id': 307, 'name': 'Zlatko Dalić', 'nickname...",776,Denmark,male,NaN,61,Denmark,"[{'id': 641, 'name': 'Åge Fridtjof Hareide', '...",1.0.2,33,Round of 16,4263.0,Stadion Nizhny Novgorod,188.0,Russia,730.0,N. Pitana,NaN,NaN,43,3,International,FIFA World Cup,male,2018,20

In [38]:
df_events_matches_competitions.shape

(227886, 168)

##### Export DataFrame

In [39]:
# Export DataFrame as a CSV file
if not os.path.exists(os.path.join(data_dir_sb, 'raw', 'combined', 'combined_wc2018.csv')):
    df_events_matches_competitions.to_csv(os.path.join(data_dir_sb, 'raw', 'combined', 'combined_wc2018.csv'), index=None, header=True)
else:
    pass

### <a id='#section3.5'>3.5. Initial Data Handling</a>
Let's quality of the dataset by looking first and last rows in pandas using the [head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) and [tail()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html) methods.

In [40]:
# Display the first five rows of the DataFrame, df_events_matches_competitions
df_events_matches_competitions.head()

,level_0,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,location,player.id,player.name,position.id,position.name,pass.recipient.id,pass.recipient.name,pass.length,pass.angle,pass.height.id,pass.height.name,pass.end_location,pass.body_part.id,pass.body_part.name,pass.type.id,pass.type.name,under_pressure,carry.end_location,pass.outcome.id,pass.outcome.name,pass.aerial_won,duel.type.id,duel.type.name,ball_receipt.outcome.id,ball_receipt.outcome.name,pass.switch,pass.assisted_shot_id,pass.goal_assist,shot.statsbomb_xg,shot.end_location,shot.key_pass_id,shot.outcome.id,shot.outcome.name,shot.body_part.id,shot.body_part.name,shot.type.id,shot.type.name,shot.technique.id,shot.technique.name,shot.freeze_frame,goalkeeper.outcome.id,goalkeeper.outcome.name,goalkeeper.body_part.id,goalkeeper.body_part.name,goalkeeper.type.id,goalkeeper.type.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.technique.id,goalkeeper.technique.name,shot.first_time,counterpress,foul_committed.offensive,foul_won.defensive,pass.cross,goalkeeper.end_location,clearance.aerial_won,dribble.outcome.id,dribble.outcome.name,duel.outcome.id,duel.outcome.name,pass.deflected,block.offensive,block.deflection,dribble.overrun,pass.shot_assist,interception.outcome.id,interception.outcome.name,miscontrol.aerial_won,ball_recovery.recovery_failure,foul_committed.advantage,foul_won.advantage,dribble.nutmeg,shot.aerial_won,pass.backheel,50_50.outcome.id,50_50.outcome.name,ball_recovery.offensive,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,foul_committed.type.id,foul_committed.type.name,pass.through_ball,pass.technique.id,pass.technique.name,foul_committed.card.id,foul_committed.card.name,foul_committed.penalty,foul_won.penalty,match_id,pass.cut_back,injury_stoppage.in_chain,shot.one_on_one,block.save_block,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.open_goal,shot.deflected,shot.redirect,shot.follows_dribble,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,0,0aa135b8-37b4-4482-adc7-f02e85a19bec,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,785,Croatia,1,Regular Play,785,Croatia,4141.0,"[{'player': {'id': 3444, 'name': 'Danijel Suba...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-01,20:00:00.000,1,1,available,2020-07-29T05:00,4,43,International,FIFA World Cup,3,2018,785,Croatia,male,NaN,56,Croatia,"[{'id': 307, 'name': 'Zlatko Dalić', 'nickname...",776,Denmark,male,NaN,61,Denmark,"[{'id': 641, 'name': 'Åge Fridtjof Hareide', '...",1.0.2,33,Round of 16,4263.0,Stadion Nizhny Novgorod,188.0,Russia,730.0,N. Pitana,NaN,NaN,43,3,International,FIFA World Cup,male,2018,20

In [41]:
# Display the last five rows of the DataFrame, df_events_matches_competitions
df_events_matches_competitions.tail()

,level_0,id,index,period,timestamp,minute,second,possession,duration,type.id,type.name,possession_team.id,possession_team.name,play_pattern.id,play_pattern.name,team.id,team.name,tactics.formation,tactics.lineup,related_events,location,player.id,player.name,position.id,position.name,pass.recipient.id,pass.recipient.name,pass.length,pass.angle,pass.height.id,pass.height.name,pass.end_location,pass.body_part.id,pass.body_part.name,pass.type.id,pass.type.name,under_pressure,carry.end_location,pass.outcome.id,pass.outcome.name,pass.aerial_won,duel.type.id,duel.type.name,ball_receipt.outcome.id,ball_receipt.outcome.name,pass.switch,pass.assisted_shot_id,pass.goal_assist,shot.statsbomb_xg,shot.end_location,shot.key_pass_id,shot.outcome.id,shot.outcome.name,shot.body_part.id,shot.body_part.name,shot.type.id,shot.type.name,shot.technique.id,shot.technique.name,shot.freeze_frame,goalkeeper.outcome.id,goalkeeper.outcome.name,goalkeeper.body_part.id,goalkeeper.body_part.name,goalkeeper.type.id,goalkeeper.type.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.technique.id,goalkeeper.technique.name,shot.first_time,counterpress,foul_committed.offensive,foul_won.defensive,pass.cross,goalkeeper.end_location,clearance.aerial_won,dribble.outcome.id,dribble.outcome.name,duel.outcome.id,duel.outcome.name,pass.deflected,block.offensive,block.deflection,dribble.overrun,pass.shot_assist,interception.outcome.id,interception.outcome.name,miscontrol.aerial_won,ball_recovery.recovery_failure,foul_committed.advantage,foul_won.advantage,dribble.nutmeg,shot.aerial_won,pass.backheel,50_50.outcome.id,50_50.outcome.name,ball_recovery.offensive,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,foul_committed.type.id,foul_committed.type.name,pass.through_ball,pass.technique.id,pass.technique.name,foul_committed.card.id,foul_committed.card.name,foul_committed.penalty,foul_won.penalty,match_id,pass.cut_back,injury_stoppage.in_chain,shot.one_on_one,block.save_block,pass.miscommunication,bad_behaviour.card.id,bad_behaviour.card.name,shot.open_goal,shot.deflected,shot.redirect,shot.follows_dribble,match_date,kick_off,home_score,away_score,match_status,last_updated,match_week,competition.competition_id,competition.country_name,competition.competition_name,season.season_id,season.season_name,home_team.home_team_id,home_team.home_team_name,home_team.home_team_gender,home_team.home_team_group,home_team.country.id,home_team.country.name,home_team.managers,away_team.away_team_id,away_team.away_team_name,away_team.away_team_gender,away_team.away_team_group,away_team.country.id,away_team.country.name,away_team.managers,metadata.data_version,competition_stage.id,competition_stage.name,stadium.id,stadium.name,stadium.country.id,stadium.country.name,referee.id,referee.name,referee.country.id,referee.country.name,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
227881,3545,811fde39-ca0a-4b50-9e93-73518b894ab0,3546,2,00:48:55.253,93,55,169,0.667,43,Carry,777,Tunisia,4,From Throw In,768,England,NaN,NaN,"['b90898db-1dd6-4ea9-bd19-f879bca67173', 'fec4...","[81.0, 72.0]",3308.0,Kieran Trippier,12.0,Right Midfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[82.0, 72.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-06-18,20:00:00.000,1,2,available,2020-07-29T05:00,1,43,International,FIFA World Cup,3,2018,777,Tunisia,male,Group G,232,Tunisia,"[{'id': 557, 'name': 'Nabil Maâloul', 'nicknam...",768,England,male,Group G,68,England,"[{'id': 277, 'name': 'Gareth Southgate', 'nick...",1.0.2,10,Group Stage,4257.0,Volgograd Arena,188.0,Russia,175.0,W. Rold

In [42]:
# Print the shape of the DataFrame, df_events_matches_competitions
print(df_events_matches_competitions.shape)

(227886, 168)


In [43]:
# Print the column names of the DataFrame, df_events_matches_competitions
print(df_events_matches_competitions.columns)

Index(['level_0', 'id', 'index', 'period', 'timestamp', 'minute', 'second',
       'possession', 'duration', 'type.id',
       ...
       'referee.country.id', 'referee.country.name', 'competition_id',
       'season_id', 'country_name', 'competition_name', 'competition_gender',
       'season_name', 'match_updated', 'match_available'],
      dtype='object', length=168)


The joined dataset has forty features (columns). Full details of these attributes can be found in the [Data Dictionary](section3.3.1).

In [44]:
# Data types of the features of the raw DataFrame, df_events_matches_competitions
df_events_matches_competitions.dtypes

level_0                int64
id                    object
index                  int64
period                 int64
timestamp             object
                       ...  
competition_name      object
competition_gender    object
season_name            int64
match_updated         object
match_available       object
Length: 168, dtype: object

Full details of these attributes and their data types can be found in the [Data Dictionary](section3.3.1).

In [45]:
# Info for the raw DataFrame, df_events_matches_competitions
#df_events_matches_competitions.info()    # commented out as it crashes the notebook

In [46]:
# Description of the raw DataFrame, df_events_matches_competitions, showing some summary statistics for each numberical column in the DataFrame
#df_events_matches_competitions.describe()    # commented out as it crashes the notebook

In [47]:
# Plot visualisation of the missing values for each feature of the raw DataFrame, df_events_matches_competitions
#msno.matrix(df_events_matches_competitions, figsize = (30, 7))    # commented out as it crashes the notebook

In [48]:
# Counts of missing values
null_value_stats = df_events_matches_competitions.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

duration                 84963
tactics.formation       227642
tactics.lineup          227642
related_events           11167
location                  3810
                         ...  
stadium.country.name      7140
referee.id                7140
referee.name              7140
referee.country.id      103266
referee.country.name    103266
Length: 116, dtype: int64

---

<a id='section4'></a>

## <a id='#section4'>4. Data Engineering</a>
Before conducting an [Exploratory Data Analysis (EDA)](#section5) of the data, we'll first need to clean and wrangle the datasets to a form that meet our needs.

### <a id='#section4.1'>4.1. Assign Raw DataFrame to Engineered DataFrame</a>

In [49]:
# Assign Raw DataFrame to Engineered DataFrame
df_sb = df_events_matches_competitions

In [50]:
df_sb['Full_Fixture_Date'] = df_sb['match_date'].astype(str) + ' ' + df_sb['home_team.home_team_name'].astype(str)  + ' ' + df_sb['home_score'].astype(str) + ' ' + ' vs. ' + ' ' + df_sb['away_score'].astype(str) + ' ' + df_sb['away_team.away_team_name'].astype(str)

### <a id='#section4.2'>4.2. Sort DataFrame</a>

In [51]:
df_sb = df_sb.sort_values(['Full_Fixture_Date', 'match_date', 'timestamp'], ascending=[True, True, True])

### <a id='#section4.3'>4.3. Create New Attributes</a>

In [53]:
#

##
df_sb['Team'] = np.where(df_sb['team.name'] == df_sb['home_team.home_team_name'], df_sb['home_team.home_team_name'], df_sb['away_team.away_team_name'])
df_sb['Opponent'] = np.where(df_sb['team.name'] == df_sb['away_team.away_team_name'], df_sb['home_team.home_team_name'], df_sb['away_team.away_team_name'])


##
df_sb['next_event'] = df_sb['type.name'].shift(-1)
df_sb['previous_event'] = df_sb['type.name'].shift(+1)
df_sb['next_team_possession'] = df_sb['possession_team.name'].shift(-1)
df_sb['previous_team_possession'] = df_sb['possession_team.name'].shift(+1)
df_sb['possession_retained'] = np.where((df_sb['possession_team.name'] == df_sb['next_team_possession']), 1, 0)
df_sb['xA'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['next_event'] == 'Shot')), df_sb['shot.statsbomb_xg'], 0)
df_sb['key_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['next_event'] == 'Shot')), 1, 0)


##

###
df_sb['location'] = df_sb['location'].astype(str)
df_sb['pass.end_location'] = df_sb['pass.end_location'].astype(str)
df_sb['carry.end_location'] = df_sb['carry.end_location'].astype(str)
df_sb['shot.end_location'] = df_sb['shot.end_location'].astype(str)
df_sb['goalkeeper.end_location'] = df_sb['goalkeeper.end_location'].astype(str)
df_sb['shot.end_location'] = df_sb['shot.end_location'].astype(str)

###
df_sb['location'] = df_sb['location'].str.replace('[','')
df_sb['pass.end_location'] = df_sb['pass.end_location'].str.replace('[','')
df_sb['carry.end_location'] = df_sb['carry.end_location'].str.replace('[','')
df_sb['shot.end_location'] = df_sb['shot.end_location'].str.replace('[','')
df_sb['goalkeeper.end_location'] = df_sb['goalkeeper.end_location'].str.replace('[','')

###
df_sb['location'] = df_sb['location'].str.replace(']','')
df_sb['pass.end_location'] = df_sb['pass.end_location'].str.replace(']','')
df_sb['carry.end_location'] = df_sb['carry.end_location'].str.replace(']','')
df_sb['shot.end_location'] = df_sb['shot.end_location'].str.replace(']','')
df_sb['goalkeeper.end_location'] = df_sb['goalkeeper.end_location'].str.replace(']','')

###
df_sb['location_x'], df_sb['location_y'] = df_sb['location'].str.split(',', 1).str
df_sb['pass.end_location_x'], df_sb['pass.end_location_y'] = df_sb['pass.end_location'].str.split(',', 1).str
df_sb['carry.end_location_x'], df_sb['carry.end_location_y'] = df_sb['carry.end_location'].str.split(',', 1).str
df_sb['shot.end_location_x'], df_sb['shot.end_location_y'], df_sb['shot.end_location_z'] = df_sb['shot.end_location'].str.split(',', 3).str[0:3].str
df_sb['goalkeeper.end_location_x'], df_sb['goalkeeper.end_location_y'] = df_sb['goalkeeper.end_location'].str.split(',', 1).str


##
df_sb['complete_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.outcome.name'] == 'Complete')), 1, 0)
df_sb['incomplete_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.outcome.name'] == 'Incomplete')), 1, 0)
df_sb['attempted_pass'] = df_sb['complete_pass'] + df_sb['incomplete_pass']
df_sb['complete_pressured_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['under_pressure'] == True) & (df_sb['pass.outcome.name'] == 'Complete')), 1, 0)
df_sb['incomplete_pressured_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['under_pressure'] == True) & (df_sb['pass.outcome.name'] == 'Incomplete')), 1, 0)
df_sb['attempted_pressured_pass'] = df_sb['complete_pressured_pass'] + df_sb['incomplete_pressured_pass']
df_sb['complete_pass_distance'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['complete_pass'] == 1)), df_sb['pass.length'], 0)
df_sb['progressive_pass'] = np.where((df_sb['type.name'] == 'Pass') & (df_sb['pass.angle'] > 0), 1, 0)
df_sb['progressive_pass_distance'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['complete_pass'] == 1) & (df_sb['progressive_pass'] == 1)), df_sb['pass.length'], 0)
df_sb['completed_short_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['complete_pass'] == 1) & (df_sb['pass.length'] >= 5) & (df_sb['pass.length'] <= 15)), 1, 0)
df_sb['attempted_short_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.length'] >= 5) & (df_sb['pass.length'] <= 15)), 1, 0)
df_sb['completed_medium_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['complete_pass'] == 1) & (df_sb['pass.length'] >= 15) & (df_sb['pass.length'] <= 30)), 1, 0)
df_sb['attempted_medium_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.length'] > 15) & (df_sb['pass.length'] <= 30)), 1, 0)
df_sb['completed_long_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['complete_pass'] == 1) & (df_sb['pass.length'] > 30)), 1, 0)
df_sb['attempted_long_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.length'] > 30)), 1, 0)
df_sb['assisted_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.goal_assist'] == True)), 1, 0)
df_sb['xA'] = np.where(df_sb['next_event'] == 'Shot', df_sb['shot.statsbomb_xg'], 0)    # fix expected assists
df_sb['tackle'] = np.where(((df_sb['type.name'] == 'Duel') & (df_sb['duel.type.name'] == 'Tackle')), 1, 0)
df_sb['interception'] = np.where(df_sb['type.name'] == 'Interception', 1, 0)
df_sb['dribbled_past'] = np.where(df_sb['type.name'] == 'Dribbled Past', 1, 0)
df_sb['open_play_pass'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['play_pattern.name'] == 'Regular Play')), 1, 0) 
df_sb['carry_attempted'] = np.where(df_sb['type.name'] == 'Carry', 1, 0)
df_sb['carry_completed'] = np.where(((df_sb['type.name'] == 'Carry') & (df_sb['possession_retained'] == 1)), 1, 0)
df_sb['carry_length'] = np.where(df_sb['type.name'] == 'Carry', np.sqrt((df_sb['location_y'] - df_sb['carry.end_location_y']) ** 2 + (df_sb['location_x'] - df_sb['carry.end_location_x']) ** 2), 0)
df_sb['clearance'] = np.where(df_sb['type.name'] == 'Clearance', 1, 0)
#df_sb['aerial_duel_won'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.body_part.name'] == 'Head')), 1, 0)    # check this one, not so sure
df_sb['aerial_duel_won'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.aerial_won'] == True)), 1, 0)    # check this one, not so sure
df_sb['aerial_duel_lost'] = np.where(((df_sb['type.name'] == 'Duel') & (df_sb['duel.type.name'] == 'Aerial Lost')), 1, 0)    # check this one, not so sure
df_sb['pressured_long_ball'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['under_pressure'] == True) & (df_sb['pass.height.name'] == 'High Pass')), 1, 0)
df_sb['unpressured_long_ball'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['under_pressure'] == False) & (df_sb['pass.height.name'] == 'High Pass')), 1, 0)
df_sb['pressure'] = np.where(df_sb['type.name'] == 'Pressure', 1, 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single charact

TypeError: unsupported operand type(s) for -: 'str' and 'str'

### <a id='#section4.4'>4.4. Fill Null Values</a>
Certain fields only contain values if a there were unsuccessful i.e. an incomplete pass. Fill these attributes with the relevant values to makes the data easier to work with

In [55]:
df_sb['pass.outcome.name'] = np.where(((df_sb['type.name'] == 'Pass') & (df_sb['pass.outcome.name'] != 'Incomplete')), df_sb['pass.outcome.name'].fillna('Complete'), np.nan)
#df_sb['pass.outcome.name'] = df_sb['pass.outcome.name'].fillna('Complete')
df_sb['under_pressure'] = df_sb['under_pressure'].fillna(False)

### <a id='#section4.5'>4.5. Determine Each Player's Most Frequent Position</a>

In [ ]:
# Determine Each Player's Most Frequeny Position

## Groupby and Aggregate by player name and position
df_sb_player_positions = (df_sb
                              .groupby(['player.name', 'position.name'])
                              .agg({'type.name': 'count'})
                              .reset_index()
                         )

## Rename columns after groupby and aggregation
df_sb_player_positions.columns = ['player', 'position', 'count']

## Take the high occuring position per player
df_sb_player_positions = (df_sb_player_positions
                              .groupby(['player', 'position'])
                              .apply(lambda x: x.sort_values(['count'], ascending=False))
                              .reset_index(drop=True)
                              .groupby(['player']).head(1)
                              .drop(['count'], axis=1)
                         )
## Display DataFrame
df_sb_player_positions.head(5)

---

<a id='section5'></a>

## <a id='#section5'>5. Aggregate Data</a>

Twelve key metrics for player assessment and their definitions are as follows:
*    **Tackles & Dribbles Past p90**: ...
*    **Dribblers tackled %**: the number of dribblers tackled divided by dribblers tackled plus times dribbled past;
*    **Aerial Wins %**: the percentage of aerial battles won divided by total aerial battles;
*    **Aerial Wins p90**: the number of aerial duels a player wins, per 90 minutes;
*    **Open Play Passes p90**: the number of attempted passes in open play, per 90 minutes;
*    **Pass Completion %**: the number of completed passes divded by the number of attempted passes;
*    **Being Pressured Change in Pass %**: How does passing % change when under pressure? This is calculated as Pressured Pass % minus Pass %
*    **Deep Progressions p90**: the number of passes and dribbles/carries into the opposition final third, p90
*    **xGBuildup p90**: xG Chain is the total xG of every possession the player is involved in. xG build up is the same minus shots and key passes. To determine this: 1.Find all the possessions each player is involved in, 2.Find all the shots within those possessions, 3.Sum their xG (you might take the highest xG per possession, or you might treat the shots as dependent events), and 4.Assign that sum to each player, however involved they were.
*    **Carries p90**: the number of carries, defined as when a player controls the ball at their feet while moving or standing still, p90;
*    **Carry %**: percentage of a player's Carries that were successful; and
*    **Carry Length p90**: average Carry length, p90.

Other metrics considered for defenders:
*    **Fouls p90**: the number of fouls per 90 minutes
*    **Pressures p90**: the number of times applying pressure to opposing player who is receiving, carrying or releasing the ball
*    **Pressured Long Balls**: 
*    **Unpressured Long Balls**: the number of completed long balls while not under pressure per 90.
*    **Tackles p90**: the number of tackles per 90 minutes (ideally this would be pAdj Tackles, the number of tackles adjusted proportionally to the possession volume of a team. Unfortunately, in the time available for this task, this is difficult to determine);
*    **Interceptions**: the number of interceptions per 90 minutes (ideally this would be pAdj Interceptions, the number of interceptions adjusted proportionally to the possession volume of a team. Unfortunately, in the time available for this task, this is difficult to determine); 
*    **Average Defensive Action Distance**: the average distance from the goal line that the player successfully makes a defensive action;
*    **Clearances p90**: the number of times a player makes a clearance or plays a long ball while under pressure, per 90 minutes;
*    **Blocks p90**: the number of blocks, per 90 minutes. A 'block' is defined as blocking the ball by standing in its path; and
*    **Blocks/Shot p90**: the number of blocks made per shot faced, per 90 minutes. 

Baseline attributes required:
*    **Player**: the player's name;
*    **Team**: the team or in this case, the country that the player is playing for;
*    **Opponent**: the team or in this case, the country that the player is playing against;
*    **Full Fixture Date**: ;
*    **Minutes played**: the number of minutes played; and
*    **Games played**: the total number of matches played (for the aggregated version only).

### <a id='#section5.1'>5.1. Groupby and Aggregate by Player and Match</a>

In [ ]:
# Create Passing DataFrame

## Groupby and aggregate
df_sb_match_by_match = (df_sb
                           .groupby(['player.name',
                                     'Team',
                                     'Opponent',
                                     'Full_Fixture_Date',
                                    ]
                                   )
                           .agg({'minute': ['min', 'max'],
                                 'complete_pass': ['sum'],
                                 'incomplete_pass': ['sum'],
                                 'attempted_pass': ['sum'],
                                 'complete_pressured_pass': ['sum'],
                                 'incomplete_pressured_pass': ['sum'],
                                 'attempted_pressured_pass': ['sum'],
                                 'complete_pass_distance': ['sum'],
                                 'progressive_pass_distance': ['sum', 'mean'],
                                 'completed_short_pass': ['sum'],
                                 'attempted_short_pass': ['sum'],
                                 'completed_medium_pass': ['sum'],
                                 'attempted_medium_pass': ['sum'],
                                 'completed_long_pass': ['sum'],
                                 'attempted_long_pass': ['sum'],
                                 'assisted_pass': ['sum'],
                                 'xA': ['sum'],
                                 'key_pass': ['sum'],
                                 'shot.statsbomb_xg': ['sum'],
                                 'tackle': ['sum'],
                                 'interception': ['sum'],
                                 'dribbled_past': ['sum'],
                                 'open_play_pass': ['sum'],
                                 'carry_attempted': ['sum'],
                                 'carry_completed': ['sum'],
                                 'carry_length': ['sum'],
                                 'clearance': ['sum'],
                                 'aerial_duel_won': ['sum'],
                                 'aerial_duel_lost': ['sum'],
                                 'pressured_long_ball': ['sum'],
                                 'unpressured_long_ball': ['sum'],
                                 'pressure': ['sum'],
                                 'Full_Fixture_Date': ['nunique']
                                }
                               )
                       )

## Drop level
df_sb_match_by_match.columns = df_sb_match_by_match.columns.droplevel(level=0)

## Reset index
df_sb_match_by_match = df_sb_match_by_match.reset_index()

## Rename columns
df_sb_match_by_match.columns = ['player',
                                'team',
                                'opponent',
                                'full_fixture_date',
                                'min_start',
                                'min_end',
                                'passes_completed',
                                'passes_incompleted',
                                'passes_attempted',
                                'pressured_passes_completed',
                                'pressured_passes_incompleted',
                                'pressured_passes_attempted',
                                'total_distance_completed_passes',
                                'total_distance_progressive_passes',
                                'average_distance_progressive_passes',
                                'short_passes_completed',
                                'short_passes_attempted',
                                'medium_passes_completed',
                                'medium_passes_attempted',
                                'long_passes_completed',
                                'long_passes_attempted',
                                'assists',
                                'expected_assists',
                                'key_passes',
                                'expected_goals',
                                'tackles',
                                'interceptions',
                                'dribbled_past',
                                'open_play_passes',
                                'carries',
                                'carries_completed',
                                'carry_length',
                                'clearances',
                                'aerial_duels_won',
                                'aerial_duels_lost',
                                'pressured_long_balls',
                                'unpressured_long_balls',
                                'pressures',
                                'games_played'
                               ]

## Replace values
df_sb_match_by_match['min_start'] = np.where(df_sb_match_by_match['min_start'] <= 5, 0, df_sb_match_by_match['min_start'])    # fix this

## Create new attributes post-aggregation
df_sb_match_by_match['minutes'] = df_sb_match_by_match['min_end'] - df_sb_match_by_match['min_start']
df_sb_match_by_match['pass_completion_percentage'] = ((df_sb_match_by_match['passes_completed'] / (df_sb_match_by_match['passes_completed'] + df_sb_match_by_match['passes_incompleted'])) * 100).round(1)
df_sb_match_by_match['pressured_pass_completion_percentage'] = ((df_sb_match_by_match['pressured_passes_completed'] / (df_sb_match_by_match['pressured_passes_completed'] + df_sb_match_by_match['pressured_passes_incompleted'])) * 100).round(1)
df_sb_match_by_match['pressure_change_in_pass_completion_percentage'] = df_sb_match_by_match['pressured_pass_completion_percentage'] - df_sb_match_by_match['pass_completion_percentage'].round(1) 
df_sb_match_by_match['short_pass_completion_percentage'] = ((df_sb_match_by_match['short_passes_completed'] / df_sb_match_by_match['short_passes_attempted']) * 100).round(1)
df_sb_match_by_match['medium_pass_completion_percentage'] = ((df_sb_match_by_match['medium_passes_completed'] / df_sb_match_by_match['medium_passes_attempted']) * 100).round(1)
df_sb_match_by_match['long_pass_completion_percentage'] = ((df_sb_match_by_match['long_passes_completed'] / df_sb_match_by_match['long_passes_attempted']) * 100).round(1)
df_sb_match_by_match['assist_minus_expected_assist'] = df_sb_match_by_match['assists'] - df_sb_match_by_match['expected_assists']
df_sb_match_by_match['tackles_and_interceptions'] = df_sb_match_by_match['tackles'] + df_sb_match_by_match['interceptions']
df_sb_match_by_match['tackles_and_dribbles_past'] = (df_sb_match_by_match['tackles'] / (df_sb_match_by_match['tackles'] + df_sb_match_by_match['dribbled_past']) * 100).round(1)
df_sb_match_by_match['carry_completion_percentage'] = ((df_sb_match_by_match['carries_completed'] / df_sb_match_by_match['carries']) * 100).round(1)
df_sb_match_by_match['aerial_duels_win_percentage'] = ((df_sb_match_by_match['aerial_duels_won'] / (df_sb_match_by_match['aerial_duels_won'] + df_sb_match_by_match['aerial_duels_lost'])) * 100).round(1)

## Create 'per 90' metrics
df_sb_match_by_match['tackles_and_interceptions_p90'] = (df_sb_match_by_match['tackles_and_interceptions'] / df_sb_match_by_match['minutes']) * 90
df_sb_match_by_match['open_play_passes_p90'] = (df_sb_match_by_match['open_play_passes'] / df_sb_match_by_match['minutes']) * 90
df_sb_match_by_match['carries_p90'] = (df_sb_match_by_match['carries'] / df_sb_match_by_match['minutes']) * 90
df_sb_match_by_match['carry_length_p90'] = (df_sb_match_by_match['carry_length'] / df_sb_match_by_match['minutes']) * 90
df_sb_match_by_match['aerial_duels_won_p90'] = (df_sb_match_by_match['aerial_duels_won'] / df_sb_match_by_match['minutes']) * 90
df_sb_match_by_match['pressured_long_balls_p90'] = (df_sb_match_by_match['pressured_long_balls'] / df_sb_match_by_match['minutes']) * 90
df_sb_match_by_match['unpressured_long_balls_p90'] = (df_sb_match_by_match['unpressured_long_balls'] / df_sb_match_by_match['minutes']) * 90
df_sb_match_by_match['pressures_p90'] = (df_sb_match_by_match['pressures'] / df_sb_match_by_match['minutes']) * 90

## Remove columns
df_sb_match_by_match = df_sb_match_by_match.drop(['min_start', 'min_end'], axis=1)

## Round figures
df_sb_match_by_match['total_distance_progressive_passes'] = df_sb_match_by_match['total_distance_progressive_passes'].round(1)
df_sb_match_by_match['average_distance_progressive_passes'] = df_sb_match_by_match['average_distance_progressive_passes'].round(1)
df_sb_match_by_match['total_distance_completed_passes'] = df_sb_match_by_match['total_distance_completed_passes'].round(0)
df_sb_match_by_match['expected_goals'] = df_sb_match_by_match['expected_goals'].round(3)
df_sb_match_by_match['tackles_and_interceptions_p90'] = df_sb_match_by_match['tackles_and_interceptions_p90'].round(2)
df_sb_match_by_match['open_play_passes_p90'] = df_sb_match_by_match['open_play_passes_p90'].round(1)
df_sb_match_by_match['carries_p90'] = df_sb_match_by_match['carries_p90'].round(1)
df_sb_match_by_match['carry_length_p90'] = df_sb_match_by_match['carry_length_p90'].round(1)
df_sb_match_by_match['aerial_duels_won_p90'] = df_sb_match_by_match['aerial_duels_won_p90'].round(1)
df_sb_match_by_match['pressured_long_balls_p90'] = df_sb_match_by_match['pressured_long_balls_p90'].round(1)
df_sb_match_by_match['unpressured_long_balls_p90'] = df_sb_match_by_match['unpressured_long_balls_p90'].round(1)
df_sb_match_by_match['pressures_p90'] = df_sb_match_by_match['pressures_p90'].round(1)

## Join player positions to DataFrame
df_sb_match_by_match = pd.merge(df_sb_match_by_match, df_sb_player_positions, how='left', left_on=['player'], right_on=['player'])

## Display DataFrame
df_sb_match_by_match.head(10)

### <a id='#section5.2'>5.2. Groupby and Aggregate by Player for the Entire Tournament</a>
*    Replace `df_sb_match_by_match` for `df_sb_all`.
*    Comment out #'Opponent' and #'Full_Fixture_Date',

In [ ]:
# Create Passing DataFrame

## Groupby and aggregate
df_sb_all = (df_sb
                   .groupby(['player.name',
                             'Team',
                            ]
                           )
                   .agg({'minute': ['min', 'max'],
                         'complete_pass': ['sum'],
                         'incomplete_pass': ['sum'],
                         'attempted_pass': ['sum'],
                         'complete_pressured_pass': ['sum'],
                         'incomplete_pressured_pass': ['sum'],
                         'attempted_pressured_pass': ['sum'],
                         'complete_pass_distance': ['sum'],
                         'progressive_pass_distance': ['sum', 'mean'],
                         'completed_short_pass': ['sum'],
                         'attempted_short_pass': ['sum'],
                         'completed_medium_pass': ['sum'],
                         'attempted_medium_pass': ['sum'],
                         'completed_long_pass': ['sum'],
                         'attempted_long_pass': ['sum'],
                         'assisted_pass': ['sum'],
                         'xA': ['sum'],
                         'key_pass': ['sum'],
                         'shot.statsbomb_xg': ['sum'],
                         'tackle': ['sum'],
                         'interception': ['sum'],
                         'dribbled_past': ['sum'],
                         'open_play_pass': ['sum'],
                         'carry_attempted': ['sum'],
                         'carry_completed': ['sum'],
                         'carry_length': ['sum'],
                         'clearance': ['sum'],
                         'aerial_duel_won': ['sum'],
                         'aerial_duel_lost': ['sum'],
                         'pressured_long_ball': ['sum'],
                         'unpressured_long_ball': ['sum'],
                         'pressure': ['sum'],
                         'Full_Fixture_Date': ['nunique']
                        }
                       )
               )

## Drop level
df_sb_all.columns = df_sb_all.columns.droplevel(level=0)

## Reset index
df_sb_all = df_sb_all.reset_index()

## Rename columns
df_sb_all.columns = ['player',
                    'team',
                    'min_start',
                    'min_end',
                    'passes_completed',
                    'passes_incompleted',
                    'passes_attempted',
                    'pressured_passes_completed',
                    'pressured_passes_incompleted',
                    'pressured_passes_attempted',
                    'total_distance_completed_passes',
                    'total_distance_progressive_passes',
                    'average_distance_progressive_passes',
                    'short_passes_completed',
                    'short_passes_attempted',
                    'medium_passes_completed',
                    'medium_passes_attempted',
                    'long_passes_completed',
                    'long_passes_attempted',
                    'assists',
                    'expected_assists',
                    'key_passes',
                    'expected_goals',
                    'tackles',
                    'interceptions',
                    'dribbled_past',
                    'open_play_passes',
                    'carries',
                    'carries_completed',
                    'carry_length',
                    'clearances',
                    'aerial_duels_won',
                    'aerial_duels_lost',
                    'pressured_long_balls',
                    'unpressured_long_balls',
                    'pressures',
                    'games_played'
                   ]

## Replace values
df_sb_all['min_start'] = np.where(df_sb_all['min_start'] <= 5, 0, df_sb_all['min_start'])    # fix this

## Create new attributes post-aggregation
df_sb_all['minutes'] = df_sb_all['min_end'] - df_sb_all['min_start']
df_sb_all['pass_completion_percentage'] = ((df_sb_all['passes_completed'] / (df_sb_all['passes_completed'] + df_sb_all['passes_incompleted'])) * 100).round(1)
df_sb_all['pressured_pass_completion_percentage'] = ((df_sb_all['pressured_passes_completed'] / (df_sb_all['pressured_passes_completed'] + df_sb_all['pressured_passes_incompleted'])) * 100).round(1)
df_sb_all['pressure_change_in_pass_completion_percentage'] = df_sb_all['pressured_pass_completion_percentage'] - df_sb_all['pass_completion_percentage'].round(1) 
df_sb_all['short_pass_completion_percentage'] = ((df_sb_all['short_passes_completed'] / df_sb_all['short_passes_attempted']) * 100).round(1)
df_sb_all['medium_pass_completion_percentage'] = ((df_sb_all['medium_passes_completed'] / df_sb_all['medium_passes_attempted']) * 100).round(1)
df_sb_all['long_pass_completion_percentage'] = ((df_sb_all['long_passes_completed'] / df_sb_all['long_passes_attempted']) * 100).round(1)
df_sb_all['assist_minus_expected_assist'] = df_sb_all['assists'] - df_sb_all['expected_assists']
df_sb_all['tackles_and_interceptions'] = df_sb_all['tackles'] + df_sb_all['interceptions']
df_sb_all['tackles_and_dribbles_past'] = (df_sb_all['tackles'] / (df_sb_all['tackles'] + df_sb_all['dribbled_past']) * 100).round(1)
df_sb_all['carry_completion_percentage'] = ((df_sb_all['carries_completed'] / df_sb_all['carries']) * 100).round(1)
df_sb_all['aerial_duels_win_percentage'] = ((df_sb_all['aerial_duels_won'] / (df_sb_all['aerial_duels_won'] + df_sb_all['aerial_duels_lost'])) * 100).round(1)

## Create 'per 90' metrics
df_sb_all['tackles_and_interceptions_p90'] = (df_sb_all['tackles_and_interceptions'] / df_sb_all['minutes']) * 90
df_sb_all['open_play_passes_p90'] = (df_sb_all['open_play_passes'] / df_sb_all['minutes']) * 90
df_sb_all['carries_p90'] = (df_sb_all['carries'] / df_sb_all['minutes']) * 90
df_sb_all['carry_length_p90'] = (df_sb_all['carry_length'] / df_sb_all['minutes']) * 90
df_sb_all['aerial_duels_won_p90'] = (df_sb_all['aerial_duels_won'] / df_sb_all['minutes']) * 90
df_sb_all['pressured_long_balls_p90'] = (df_sb_all['pressured_long_balls'] / df_sb_all['minutes']) * 90
df_sb_all['unpressured_long_balls_p90'] = (df_sb_all['unpressured_long_balls'] / df_sb_all['minutes']) * 90
df_sb_all['pressures_p90'] = (df_sb_all['pressures'] / df_sb_all['minutes']) * 90

## Remove columns
df_sb_all = df_sb_all.drop(['min_start', 'min_end'], axis=1)

## Round figures
df_sb_all['total_distance_progressive_passes'] = df_sb_all['total_distance_progressive_passes'].round(1)
df_sb_all['average_distance_progressive_passes'] = df_sb_all['average_distance_progressive_passes'].round(1)
df_sb_all['total_distance_completed_passes'] = df_sb_all['total_distance_completed_passes'].round(0)
df_sb_all['expected_goals'] = df_sb_all['expected_goals'].round(3)
df_sb_all['tackles_and_interceptions_p90'] = df_sb_all['tackles_and_interceptions_p90'].round(2)
df_sb_all['open_play_passes_p90'] = df_sb_all['open_play_passes_p90'].round(1)
df_sb_all['carries_p90'] = df_sb_all['carries_p90'].round(1)
df_sb_all['carry_length_p90'] = df_sb_all['carry_length_p90'].round(1)
df_sb_all['aerial_duels_won_p90'] = df_sb_all['aerial_duels_won_p90'].round(1)
df_sb_all['pressured_long_balls_p90'] = df_sb_all['pressured_long_balls_p90'].round(1)
df_sb_all['unpressured_long_balls_p90'] = df_sb_all['unpressured_long_balls_p90'].round(1)
df_sb_all['pressures_p90'] = df_sb_all['pressures_p90'].round(1)

## Join player positions to DataFrame
df_sb_all = pd.merge(df_sb_all, df_sb_player_positions, how='left', left_on=['player'], right_on=['player'])

## Display DataFrame
df_sb_all.head(10)

### <a id='#section5.3'>5.3. Export Data</a>

In [ ]:
# Export 
df_sb_match_by_match.to_csv(data_dir + '/export/' + 'sb_wc2018_events_agg_match_by_match.csv', index=None, header=True)
df_sb_all.to_csv(data_dir + '/export/' + 'sb_wc2018_events_agg_all.csv', index=None, header=True)

---

<a id='section6'></a>

## <a id='#section6'>6. Subset Data</a>
The following code creates DataFrames for additional Tableau visualisation that are not part of the submission for this task.

### <a id='#section6.1'>6.1. Extract Lineups from DataFrame</a>

In [ ]:
# List unique values in the df_sb['type.name'] column
df_sb['type.name'].unique()

The starting XI players and formation can be found in the rows where `type.name` is 'Starting XI'.

In [ ]:
df_lineup = df_sb[df_sb['type.name'] == 'Starting XI']

In [ ]:
df_lineup

In [ ]:
# Streamline DataFrame to include just the columns of interest

## Define columns
cols = ['id', 'type.name', 'match_date', 'kick_off', 'Full_Fixture_Date', 'team.id', 'team.name', 'tactics.formation', 'tactics.lineup', 'competition_name', 'season_name', 'home_team.home_team_name', 'away_team.away_team_name', 'Team', 'Opponent', 'home_score', 'away_score']

## Select only columns of interest
df_lineup_select = df_lineup[cols]

In [ ]:
df_lineup_select

We can see from the extracted lineup data so far. To get the stating XI players, we need to breakdown the `tactics.lineup` attribute.

In [ ]:
# Normalize tactics.lineup - see: https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

## explode all columns with lists of dicts
df_lineup_select_normalize = df_lineup_select.apply(lambda x: x.explode()).reset_index(drop=True)

## list of columns with dicts
cols_to_normalize = ['tactics.lineup']

## if there are keys, which will become column names, overlap with excising column names. add the current column name as a prefix
normalized = list()

for col in cols_to_normalize:
    d = pd.json_normalize(df_lineup_select_normalize[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

## combine df with the normalized columns
df_lineup_select_normalize = pd.concat([df_lineup_select_normalize] + normalized, axis=1).drop(columns=cols_to_normalize)

## display(df_lineup_select_normalize)
df_lineup_select_normalize.head(30)

In [ ]:
df_lineup_engineered = df_lineup_select_normalize

In [ ]:
# Streamline DataFrame to include just the columns of interest

## Define columns
cols = ['id', 'match_date', 'kick_off', 'Full_Fixture_Date', 'type.name', 'season_name', 'competition_name', 'home_team.home_team_name', 'away_team.away_team_name', 'Team', 'Opponent', 'home_score', 'away_score', 'tactics.formation', 'tactics.lineup_jersey_number', 'tactics.lineup_position_id', 'tactics.lineup_player_name', 'tactics.lineup_position_name']

## Select only columns of interest
df_lineup_engineered_select = df_lineup_engineered[cols]

In [ ]:
df_lineup_engineered_select['tactics.formation'] = df_lineup_engineered_select['tactics.formation'].astype('Int64')
df_lineup_engineered_select['tactics.lineup_jersey_number'] = df_lineup_engineered_select['tactics.lineup_jersey_number'].astype('Int64')

In [ ]:
df_lineup_engineered_select.head(5)

In [ ]:
df_lineup_engineered_select.columns

In [ ]:
## Rename columns
df_lineup_engineered_select = df_lineup_engineered_select.rename(columns={'id': 'Match_Id',
                                                                          'match_date': 'Match_Date',
                                                                          'kick_off': 'Kick_Off',
                                                                          'type.name': 'Type_Name',
                                                                          'season_name': 'Season',
                                                                          'competition_name': 'Competition',
                                                                          'home_team.home_team_name': 'Home_Team',
                                                                          'away_team.away_team_name': 'Away_Team',
                                                                          'home_score': 'Home_Score',
                                                                          'away_score': 'Away_Score',
                                                                          'tactics.formation': 'Formation',
                                                                          'tactics.lineup_jersey_number': 'Shirt_Number',
                                                                          'tactics.lineup_position_id': 'Position_Number',
                                                                          'tactics.lineup_player_name': 'Player_Name',
                                                                          'tactics.lineup_position_name': 'Position_Name'
                                                                         }
                                                                         
                                                                )

## Display DataFrame
df_lineup_engineered_select.head()

In [ ]:
# Convert Match_Date from string to datetime64[ns]
df_lineup_engineered_select['Match_Date']= pd.to_datetime(df_lineup_engineered_select['Match_Date'])

In [ ]:
"""
# THIS IS NOT WORKING ATM

# Convert Kick_Off from string to datetime64[ns]
df_lineup_engineered_select['Kick_Off']= pd.to_datetime(df_lineup_engineered_select['Kick_Off'], format='%H:%M', errors='ignore')
df_lineup_engineered_select['Kick_Off'] = df_lineup_engineered_select['Kick_Off'].dt.time
"""

In [ ]:
df_lineup_engineered_select.dtypes

In [ ]:
# Put hyphens between numbers in Formation attribute

## Convert Formation attribute from Integer to String
df_lineup_engineered_select['Formation'] = df_lineup_engineered_select['Formation'].astype(str)

## Define custom function to add hyphen between letters: StackOverflow: https://stackoverflow.com/questions/29382285/python-making-a-function-that-would-add-between-letters
def f(s):
        m = s[0]
        for i in s[1:]:
             m += '-' + i
        return m
    
## Apply custom function
df_lineup_engineered_select['Formation'] = df_lineup_engineered_select.apply(lambda row: f(row['Formation']),axis=1)

In [ ]:
lst_formation = df_lineup_engineered_select['Formation'].unique().tolist()

In [ ]:
lst_formation

##### Add Position Coordinates

In [ ]:
df_formations_coords = pd.read_csv(data_dir_sb + '/sb_formation_coordinates.csv')

In [ ]:
#df_formations_coords['Id'] = df_formations_coords['Id'].astype('Int8')
#df_formations_coords['Player_Number'] = df_formations_coords['Player_Number'].astype('Int8')

In [ ]:
df_lineup_engineered_select = pd.merge(df_lineup_engineered_select, df_formations_coords, how='left', left_on=['Formation', 'Position_Number'], right_on=['Formation', 'Player_Number'])

In [ ]:
#df_lineup_engineered_select = df_lineup_engineered_select.drop(['Player_Number'], axis=1)
df_lineup_engineered_select = df_lineup_engineered_select.drop(['Id'], axis=1)
df_lineup_engineered_select = df_lineup_engineered_select.drop(['Player_Position'], axis=1)

In [ ]:
df_lineup_engineered_select.head()

##### Add Opponent Data to Each Row

In [ ]:
# Select columns of interest

## Define columns
cols = ['Match_Date',
        'Competition',
        'Full_Fixture_Date',
        'Team',
        'Formation'
       ]

##
df_lineup_opponent = df_lineup_engineered_select[cols]

##
df_lineup_opponent = df_lineup_opponent.drop_duplicates()

##
df_lineup_opponent.head()

In [ ]:
# Join DataFrame to itself on 'Date', 'Fixture', 'Team'/'Opponent', and 'Event', to join Team and Opponent together
df_lineup_engineered_opponent_select = pd.merge(df_lineup_engineered_select, df_lineup_opponent,  how='left', left_on=['Match_Date', 'Competition', 'Full_Fixture_Date', 'Opponent'], right_on = ['Match_Date', 'Competition', 'Full_Fixture_Date', 'Team'])

In [ ]:
# Clean Data

## Drop columns
df_lineup_engineered_opponent_select = df_lineup_engineered_opponent_select.drop(columns=['Team_y'])


## Rename columns
df_lineup_engineered_opponent_select = df_lineup_engineered_opponent_select.rename(columns={'Team_x': 'Team',
                                                                                            'Formation_x': 'Formation',
                                                                                            'Formation_y': 'Opponent_Formation'
                                                                                           }
                                                                                      )

## Display DataFrame
df_lineup_engineered_opponent_select.head()

##### Export DataFrame

In [ ]:
# Export 
df_lineup_engineered_opponent_select.to_csv(data_dir_sb + '/lineups/engineered/' + '/sb_lineups_wc2018.csv', index=None, header=True)

In [ ]:
# Export 
df_lineup_engineered_opponent_select.to_csv(data_dir + '/export/' + '/sb_wc2018_lineups.csv', index=None, header=True)

### <a id='#section6.2'>6.2. Tactical Shifts</a>

In [ ]:
df_tactics = df_sb[df_sb['type.name'] == 'Tactical Shift']

In [ ]:
df_tactics

In [ ]:
# Select columns of interest

##
cols = ['id', 'type.name', 'team.id', 'team.name', 'tactics.formation', 'tactics.lineup']

##
df_tactics_select = df_tactics[cols]

In [ ]:
df_tactics_select

In [ ]:
# Normalize tactics.lineup - see: https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

## explode all columns with lists of dicts
df_tactics_select_normalize = df_tactics_select.apply(lambda x: x.explode()).reset_index(drop=True)

## list of columns with dicts
cols_to_normalize = ['tactics.lineup']

## if there are keys, which will become column names, overlap with excising column names. add the current column name as a prefix
normalized = list()
for col in cols_to_normalize:
    
    d = pd.json_normalize(df_tactics_select_normalize[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

## combine df with the normalized columns
df_tactics_select_normalize = pd.concat([df_tactics_select_normalize] + normalized, axis=1).drop(columns=cols_to_normalize)

## display(df_lineup_select_normalize)
df_tactics_select_normalize.head(10)

### <a id='#section6.3'>6.3. Halves</a>

In [ ]:
df_half = df_sb[df_sb['type.name'] == 'Half Start']

In [ ]:
df_half

### <a id='#section6.4'>6.4. Isolate In-Play Events</a>
DataFrame of only player's actions i.e. removing line ups, halves, etc.

#### <a id='#section6.4.1'>6.4.1. Remove Non-Event rows</a>

In [ ]:
# List unique values in the df_sb['type.name'] column
df_sb['type.name'].unique()

In [ ]:
lst_events = ['Pass', 'Ball Receipt*', 'Carry', 'Duel', 'Miscontrol', 'Pressure', 'Ball Recovery', 'Dribbled Past', 'Dribble', 'Shot', 'Block', 'Goal Keeper', 'Clearance', 'Dispossessed', 'Foul Committed', 'Foul Won', 'Interception', 'Shield', 'Half End', 'Substitution', 'Tactical Shift', 'Injury Stoppage', 'Player Off', 'Player On', 'Offside', 'Referee Ball-Drop', 'Error']

In [ ]:
df_sb_events = df_sb[df_sb['type.name'].isin(lst_events)]

In [ ]:
df_sb_events.shape

#### <a id='#section6.4.2'>6.4.2. Break down all `location` attributes into seperate attribute for X, Y (and sometimes Z) coordinates</a>

In [ ]:
# Display all location columns
for col in df_sb_events.columns:
    if 'location' in col:
        print(col)

There are the following five 'location' attributes:
- `location`
- `pass.end_location`
- `carry.end_location`
- `shot.end_location`
- `goalkeeper.end_location`

From reviewing the official documentation [[link](https://statsbomb.com/stat-definitions/)], the five attributes have the following dimensionality:
- `location` [x, y]
- `pass.end_location` [x, y]
- `carry.end_location` [x, y]
- `shot.end_location` [x, y, z]
- `goalkeeper.end_location` [x, y]

In [ ]:
"""
# CURRENTLY NOT WORKING, NEED TO FIX

# Normalize 'shot.freeze_frame' avvtribute - see: https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

## explode all columns with lists of dicts
df_sb_events_normalize = df_sb_events.apply(lambda x: x.explode()).reset_index(drop=True)

## list of columns with dicts
cols_to_normalize = ['shot.freeze_frame']

## if there are keys, which will become column names, overlap with excising column names. add the current column name as a prefix
normalized = list()

for col in cols_to_normalize:
    d = pd.json_normalize(df_sb_events_normalize[col], sep='_')
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

## combine df with the normalized columns
df_sb_events_normalize = pd.concat([df_sb_events_normalize] + normalized, axis=1).drop(columns=cols_to_normalize)

## display(df_lineup_select_normalize)
df_sb_events_normalize.head(30)
"""

In [ ]:
df_sb_events.shape

##### Export Dataset

In [ ]:
# Export 
df_sb_events.to_csv(data_dir_sb + '/engineered/events/' + 'sb_wc2018_events.csv', index=None, header=True)

In [ ]:
# Export 
df_sb_events.to_csv(data_dir + '/export/' + 'sb_wc2018_events.csv', index=None, header=True)

#### <a id='#section6.4.3'>6.4.3. Create Passing Matrix Data</a>

In [ ]:
df1 = df_sb_events.copy()

In [ ]:
df1['df_name'] = 'df1'

In [ ]:
df1.head()

In [ ]:
df2 = df_sb_events.copy()

In [ ]:
df2['df_name'] = 'df2'

In [ ]:
df2.head()

In [ ]:
df1.head()

##### Concatanate DataFrames

In [ ]:
df_sb_events_passing = pd.concat([df1, df2])

In [ ]:
df_sb_events_passing.shape

##### ...

In [ ]:
df_sb_events_passing['Pass_X'] = np.where(df_sb_events_passing['df_name'] == 'df1', df_sb_events_passing['location_x'], df_sb_events_passing['pass.end_location_x'])
df_sb_events_passing['Pass_Y'] = np.where(df_sb_events_passing['df_name'] == 'df1', df_sb_events_passing['location_y'], df_sb_events_passing['pass.end_location_y'])

In [ ]:
df_sb_events_passing.head()

In [ ]:
sorted(df_sb_events_passing.columns)

##### Export Dataset

In [ ]:
# Export 
#df_sb_events_passing.to_csv(data_dir_sb + '/events/engineered/' + '/sb_wc2018_events_passing_matrix.csv', index=None, header=True)

# Export 
df_sb_events_passing.to_csv(data_dir + '/export/' + '/sb_wc2018_events_passing_matrix.csv', index=None, header=True)

#### <a id='#section6.4.4'>6.4.4. Create Passing Network Data</a>

See: https://community.tableau.com/s/question/0D54T00000C6YbE/football-passing-network

In [ ]:
df_sb_pass_network = df_sb_events_passing.copy()

In [ ]:
df_sb_pass_network = df_sb_pass_network[df_sb_pass_network['type.name'] == 'Pass']

In [ ]:
df_sb_pass_network['player_recipient'] = np.where(df_sb_pass_network['df_name'] == 'df1', df_sb_pass_network['player.name'], df_sb_pass_network['pass.recipient.name'])

In [ ]:
df_sb_pass_network.head()

In [ ]:
sorted(df_sb_pass_network.columns)

In [ ]:
df_sb_pass_network.shape

In [ ]:
# Select columns of interest

## Define columns
cols = ['df_name',
        'id',
        'index',
        'competition_name',
        'season_name',
        'match_date',
        'kick_off',
        'Full_Fixture_Date',
        'Team',
        'Opponent',
        'home_team.home_team_name',
        'away_team.away_team_name',
        'home_score',
        'away_score',
        'player_recipient',
        'player.name',
        'pass.recipient.name',
        'position.id',
        'position.name',
        'type.name',
        'pass.type.name',
        'pass.outcome.name',
        'location_x',
        'location_y', 
        'pass.end_location_x',
        'pass.end_location_y',
        'Pass_X',
        'Pass_Y'
       ]

##
df_sb_pass_network_select = df_sb_pass_network[cols]

In [ ]:
df_sb_pass_network_select['pass.to.from'] = df_sb_pass_network_select['player.name'] + ' - ' + df_sb_pass_network_select['pass.recipient.name']

In [ ]:
# List unique values in the df_sb_pass_network_select['pass.outcome.name'] column
df_sb_pass_network_select['pass.outcome.name'].unique()

In [ ]:
df_sb_pass_network_select = df_sb_pass_network_select[df_sb_pass_network_select['pass.outcome.name'].isnull()]

In [ ]:
df_sb_pass_network_select.shape

In [ ]:
df_sb_pass_network_select = df_sb_pass_network_select.sort_values(['season_name', 'match_date', 'kick_off', 'Full_Fixture_Date', 'index', 'id', 'df_name'], ascending=[True, True, True, True, True, True, True])

In [ ]:
df_sb_pass_network_select['Pass_X'] = df_sb_pass_network_select['Pass_X'].astype(str).astype(float)
df_sb_pass_network_select['Pass_Y'] = df_sb_pass_network_select['Pass_Y'].astype(str).astype(float)
df_sb_pass_network_select['location_x'] = df_sb_pass_network_select['location_x'].astype(str).astype(float)
df_sb_pass_network_select['location_y'] = df_sb_pass_network_select['location_y'].astype(str).astype(float)
df_sb_pass_network_select['pass.end_location_x'] = df_sb_pass_network_select['pass.end_location_x'].astype(str).astype(float)
df_sb_pass_network_select['pass.end_location_y'] = df_sb_pass_network_select['pass.end_location_y'].astype(str).astype(float)

In [ ]:
df_sb_pass_network_select.dtypes

In [ ]:
df_sb_pass_network_select.head()

In [ ]:
#

##
df_sb_pass_network_grouped = (df_sb_pass_network_select
                                  .groupby(['competition_name',
                                            'season_name',
                                            'match_date',
                                            'kick_off',
                                            'Full_Fixture_Date',
                                            'Team',
                                            'Opponent',
                                            'home_team.home_team_name',
                                            'away_team.away_team_name',
                                            'home_score',
                                            'away_score',
                                            'pass.to.from',
                                            'player.name',
                                            'pass.recipient.name',
                                            'player_recipient'
                                           ])
                                  .agg({'pass.to.from': ['count']
                                       })
                             )

##
df_sb_pass_network_grouped.columns = df_sb_pass_network_grouped.columns.droplevel(level=0)

##
df_sb_pass_network_grouped = df_sb_pass_network_grouped.reset_index()

## 
df_sb_pass_network_grouped.columns = ['competition_name',
                                      'season_name',
                                      'match_date',
                                      'kick_off',
                                      'full_fixture_date',
                                      'team',
                                      'opponent',
                                      'home_team_name',
                                      'away_team_name',
                                      'home_score',
                                      'away_score',
                                      'pass_to_from',
                                      'player_name',
                                      'pass_recipient_name',
                                      'player_recipient',
                                      'count_passes',
                                     ]

##
#df_sb_pass_network_grouped['count_passes'] = df_sb_pass_network_grouped['count_passes'] / 2

##
df_sb_pass_network_grouped = df_sb_pass_network_grouped.sort_values(['season_name', 'match_date', 'kick_off', 'full_fixture_date', 'team', 'opponent', 'pass_to_from'], ascending=[True, True, True, True, True, True, True])

##
df_sb_pass_network_grouped.head()

In [ ]:
df_sb_pass_network_grouped.shape

In [ ]:
# Select columns of interest

## Define columns
cols = ['Full_Fixture_Date',
        'player.name',
        'position.id',
        'position.name',
        'Pass_X',
        'Pass_Y'
       ]

##
df_sb_pass_network_avg_pass = df_sb_pass_network_select[cols]

In [ ]:
df_sb_pass_network_avg_pass 

In [ ]:
#

##
df_sb_pass_network_avg_pass_grouped = (df_sb_pass_network_avg_pass 
                                          .groupby(['Full_Fixture_Date',
                                                    'player.name',
                                                    'position.id',
                                                    'position.name',
                                                   ])
                                          .agg({'Pass_X': ['mean'],
                                                'Pass_Y': ['mean']
                                               })
                                     )

##
df_sb_pass_network_avg_pass_grouped.columns = df_sb_pass_network_avg_pass_grouped .columns.droplevel(level=0)

##
df_sb_pass_network_avg_pass_grouped = df_sb_pass_network_avg_pass_grouped.reset_index()

## 
df_sb_pass_network_avg_pass_grouped.columns = ['full_fixture_date',
                                               'player_name',
                                               'position_id',
                                               'position_name',
                                               'avg_location_pass_x',
                                               'avg_location_pass_y'
                                     ]

##
df_sb_pass_network_avg_pass_grouped['avg_location_pass_x'] = df_sb_pass_network_avg_pass_grouped['avg_location_pass_x'].round(decimals=1)
df_sb_pass_network_avg_pass_grouped['avg_location_pass_y'] = df_sb_pass_network_avg_pass_grouped['avg_location_pass_y'].round(decimals=1)

##
df_sb_pass_network_avg_pass_grouped = df_sb_pass_network_avg_pass_grouped.sort_values(['full_fixture_date', 'player_name'], ascending=[True, True])

##
df_sb_pass_network_avg_pass_grouped.head()

In [ ]:
# Join the Events DataFrame to the Matches DataFrame
df_sb_pass_network_final = pd.merge(df_sb_pass_network_grouped, df_sb_pass_network_avg_pass_grouped, left_on=['full_fixture_date', 'player_recipient'], right_on=['full_fixture_date', 'player_name'])

In [ ]:
## Rename columns
df_sb_pass_network_final = df_sb_pass_network_final.rename(columns={'player_name_x': 'player_name',
                                                                   #'player_name_x': 'player_name'
                                                                   }
                                                          )

In [ ]:
df_sb_pass_network_final.head()

In [ ]:
df_sb_pass_network_final.shape

##### Export Dataset

In [ ]:
# Export 
df_sb_pass_network_final.to_csv(data_dir_sb + '/engineered/events/' + 'sb_wc2018_events_passing_network.csv', index=None, header=True)

# Export 
df_sb_pass_network_final.to_csv(data_dir + '/export/' + 'sb_wc2018_events_passing_network.csv', index=None, header=True)

## <a id='#section7'>7. Summary</a>
This notebook engineers scraped [StatsBomb](https://statsbomb.com/) data using [pandas](http://pandas.pydata.org/).

## <a id='#section8'>8. Next Steps</a>
The next stage is to visualise this data in Tableau.

## <a id='#section9'>9. References</a>

#### Data
*    [StatsBomb](https://statsbomb.com/) data
*    [StatsBomb](https://github.com/statsbomb/open-data/tree/master/data) open data GitHub repository

---

***Visit my website [eddwebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)